## Hiperparametrización y entrenamiento distribuido de modelos: Hyperopt, Spark-MLlib y MLflow

### 1. MLlib

#### Carga de Datos

Se utiliza el conjunto de datos de reconocimiento de dígitos escritos a mano (MNIST). Los registros son vectores de características que representan las intensidades de los píxeles.

Este conjunto de datos se encuentra almacenado en formato libsvm en el DBFS.

In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
dbutils.fs.ls("dbfs:/databricks-datasets/mnist-digits/")

In [0]:
full_training_data = spark.read.format("libsvm").load("/databricks-datasets/mnist-digits/data-001/mnist-digits-train.txt")
test_data = spark.read.format("libsvm").load("/databricks-datasets/mnist-digits/data-001/mnist-digits-test.txt")

In [0]:
test_data.display()

In [0]:
print(f"Existen {full_training_data.count()} imágenes de entrenamiento y {test_data.count()} imágenes de prueba.")

In [0]:
training_data, validation_data = full_training_data.randomSplit([0.8, 0.2], seed=42)

In [0]:
training_data.count(), validation_data.count(), test_data.count()

### 2. MLflow

[Pyspark MLflow](https://www.mlflow.org/docs/latest/python_api/mlflow.pyspark.ml.html)

Se debe crear una función para entrenar un modelo, se define una función para entrenar un árbol de decisión cuyo objetivo es el de "envolver" el código de entrenamiento para posteriormente pasar la funcionalidad a Hyperopt e hiperparametrizar más adelante.

El algoritmo de árbol de clasificación necesita saber que las etiquetas son categorías entre 0-9, en lugar de valores continuos y para ello se utiliza la clase `StringIndexer`.

In [0]:
import mlflow

from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer

In [0]:
import mlflow.pyspark.ml

# Información del entrenamiento.
mlflow.pyspark.ml.autolog()

In [0]:
help(DecisionTreeClassifier)

In [0]:
def train_tree(minInstancesPerNode, maxBins, impurity):

    # Especifique "nested=True" ya que esto se registrará como una ejecución secundaria dentro de Hyperopt.
    with mlflow.start_run(run_name="decision_tree", nested=True):

        indexer = StringIndexer(inputCol="label", outputCol="indexedLabel")

        dtc = DecisionTreeClassifier(labelCol="indexedLabel",
                                     minInstancesPerNode=minInstancesPerNode,
                                     maxBins=maxBins,
                                     impurity=impurity)

        pipeline = Pipeline(stages=[indexer, dtc])
        model = pipeline.fit(training_data)
        evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", metricName="f1")
        predictions = model.transform(validation_data)
        validation_metric = evaluator.evaluate(predictions)
        
        mlflow.log_metric("val_f1_score", validation_metric)

    return model, validation_metric

Se recomienda ejecutar la función de entrenamiento antes de hiperparametrizar para asegurarse de que funciona correctamente.

In [0]:
initial_model, val_metric = train_tree(minInstancesPerNode=200, maxBins=2, impurity="entropy")
print(f"El árbol de decisión entrenado logró un F1-score de {val_metric} en la porción de validación")

### 3. Hyperopt

[Hyperopt](http://hyperopt.github.io/hyperopt/)

Flujo de trabajo de Hyperopt.

* Definir una función para minimizar
* Definir un espacio de búsqueda de hiperparámetros
* Especificar el algoritmo de búsqueda y utilizar `fmin()` para ajustar el modelo

In [0]:
from hyperopt import fmin, tpe, hp, STATUS_OK

Como siguientes pasos se debe definir una function a minimizar tal que reciba como valores de entrada los hiperparámetros, internamente se reutilize la función de entrenamiento definida previamente y por último que devuelva el valor de pérdida.

In [0]:
def train_with_hyperopt(hyperparams):

    # Para hiperparámetros con valores enteros, se recomienda convertirlos a tipo 'int'
    minInstancesPerNode = int(hyperparams["minInstancesPerNode"])
    maxBins = int(hyperparams["maxBins"])
    impurity = hyperparams["impurity"]

    model, f1_score = train_tree(minInstancesPerNode, maxBins, impurity)

    # Hyperopt asume el trabajo de minimizar una función de pérdida por lo que se toma el negativo del f1-score
    loss = -f1_score
    return {'loss': loss, 'status': STATUS_OK}

Se define el espacio de búsqueda sobre hiperparámetros. Este ejemplo ajusta dos hiperparámetros: `minInstancesPerNode` y `maxBins`. Consulte la [documentación de Hyperopt](https://github.com/hyperopt/hyperopt/wiki/FMin#21-parameter-expressions) para obtener más información sobre cómo definir un espacio de búsqueda y expresiones de parámetros.

In [0]:
hyperspace = {
    "minInstancesPerNode": hp.uniform("minInstancesPerNode", 10, 200),
    "maxBins": hp.uniform("maxBins", 2, 32),
    "impurity": hp.choice("impurity", ["entropy", "gini"])
}

Ajuste el modelo usando Hyperopt `fmin()`

- Establezca `max_evals` igual a el número máximo de puntos en el espacio de hiperparámetros para probar (el número máximo de modelos para ajustar y evaluar). Dado que este comando evalúa muchos modelos, puede tardar varios minutos en ejecutarse.
- También debe especificar qué algoritmo de búsqueda utilizar. Las dos opciones principales son:
   - `hyperopt.tpe.suggest`: Tree of Parzen Estimators, un enfoque bayesiano que selecciona de forma iterativa y adaptativa nuevas configuraciones de hiperparámetros para explorar en función de resultados anteriores
   - `hyperopt.rand.suggest`: Random Search, un enfoque no adaptativo que muestra aleatoriamente el espacio de búsqueda

In [0]:
search_algorithm = tpe.suggest

with mlflow.start_run():
    best_params = fmin(fn=train_with_hyperopt, space=hyperspace, algo=search_algorithm, max_evals=8)

In [0]:
# Configuración del mejor modelo
best_params

Vuelva a entrenar el modelo en el conjunto de datos con los mejores hiperparámetros encontrados.

In [0]:
best_minInstancesPerNode = int(best_params["minInstancesPerNode"])
best_minInstancesPerNode

In [0]:
best_maxBins = int(best_params["maxBins"])
best_maxBins

In [0]:
impurity = ["entropy", "gini"][best_params["impurity"]]
impurity

In [0]:
final_model, val_f1_score = train_tree(best_minInstancesPerNode, best_maxBins, impurity)

Utilice el conjunto de prueba para comparar las métricas de evaluación de los modelos inicial y "mejor".

In [0]:
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", metricName="f1")

In [0]:
import numpy as np

initial_model_test_metric = evaluator.evaluate(initial_model.transform(test_data))
final_model_test_metric = evaluator.evaluate(final_model.transform(test_data))

print(f"En los datos de prueba, el modelo inicial (sin hiperparametrización) logró un F1-score de: {np.round(initial_model_test_metric, 3)}, y el modelo final (hiperparametrizado) logró un F1-score de: {np.round(final_model_test_metric, 3)}.")